In [1]:
from codebase.classesmcmc import Data, MCMC
import  pystan
import argparse
import numpy as np
from codebase.file_utils import (
    save_obj,
    load_obj,
    make_folder,
    path_backslash
)
from codebase.ibis import essl, exp_and_normalise, run_mcmc
from scipy.special import expit
from tqdm.notebook import tqdm


##

In [11]:
log_dir = './log/20201215_095024_mcmcpseudo/'
ps = load_obj('mcmc_post_samples', log_dir)

In [12]:
import altair as alt
import pandas as pd
alt.data_transformers.disable_max_rows()

plot_data = pd.DataFrame(ps['alpha']) 
plot_data['id'] = np.arange(len(plot_data))
plot_data = plot_data.melt(id_vars=['id'], var_name=['col'])

mcmc_chart = alt.Chart(plot_data).mark_line(
    opacity = 1,
    strokeWidth = 1,
).encode(
    alt.Y('value', title=None),
    alt.X('id:O',
          title=None
         )
).properties(width=200, height=100)

(mcmc_chart).facet(
    alt.Facet('col'),
    columns=3
)


alt.FacetChart(...)

## Run full HMC Algorithm

In [38]:
exp_data = Data("1factor", 1, 200)
exp_data.generate()


In [39]:
log_dir = 'log/debugmcmc/'
fit_run = run_mcmc(
    data = exp_data.get_stan_data(),
    gen_model = False,
    model_num = 0,
    num_samples = 1000, 
    num_warmup = 1000,
    num_chains = 1,
    log_dir = log_dir,
    adapt_engaged=True
    )

In [40]:
param_names = ['beta', 'alpha', 'yy']
particles = fit_run.extract(
        permuted=False, pars=param_names)
save_obj(particles, 'ps', log_dir)

In [51]:
log_dir = 'log/debugmcmc/'
particles = load_obj('ps', log_dir)

In [41]:
np.round(np.mean(exp_data.raw_data['y'], 0), 2)

array([-0.58,  0.32, -1.44, -1.42, -1.  , -2.36])

In [42]:
samples = np.squeeze(particles['yy'])
np.round(np.mean(np.mean(samples,axis=0),axis=0), 2)

array([-0.17,  0.22, -1.34, -1.97, -0.85, -2.41])

In [43]:
for name in ['beta']:
    samples = np.squeeze(particles[name])
    print('\n\nEstimate %s'%name)
    print(np.round(np.mean(expit(samples),axis=0),2))
    print('\nRead Data')
    print(np.round(expit(exp_data.raw_data[name]), 2))



Estimate beta
[0.73 0.69 0.64 0.78 0.74 0.65]

Read Data
[0.73 0.67 0.69 0.62 0.71 0.65]


In [48]:
plot_data2 = pd.DataFrame(np.squeeze(particles['alpha'])) 
plot_data2['id'] = np.arange(len(plot_data2))
plot_data2 = plot_data2.melt(id_vars=['id'], var_name=['col'])

hmc_chart = alt.Chart(plot_data2).mark_line(
    opacity = 1,
    strokeWidth = 1,
    color='red'
).encode(
    alt.Y('value', title=None),
    alt.X('id:O',
          title=None
         )
).properties(width=200, height=100)

(hmc_chart).facet(
    alt.Facet('col'),
    columns=3
)


alt.FacetChart(...)

In [54]:
plot_data['method'] = 'mcmc1'
plot_data2['method'] = 'hmc2'
merge_plot_data = pd.concat([plot_data, plot_data2])

In [56]:
merge_chart = alt.Chart(merge_plot_data).mark_line(
    opacity = 0.6,
    strokeWidth = 1,
    color='red'
).encode(
    alt.Y('value', title=None),
    alt.X('id:O',
          title=None
         ), 
    alt.Color('method')
).properties(width=200, height=100)

(merge_chart).facet(
    alt.Facet('col'),
    columns=3
)


alt.FacetChart(...)

In [59]:
def merge_plots(ps, particles):
    plot_data = pd.DataFrame(ps) 
    plot_data['id'] = np.arange(len(plot_data))
    plot_data = plot_data.melt(id_vars=['id'], var_name=['col'])
    plot_data['method'] = 'mcmc1'

    plot_data2 = pd.DataFrame(np.squeeze(particles)) 
    plot_data2['id'] = np.arange(len(plot_data2))
    plot_data2 = plot_data2.melt(id_vars=['id'], var_name=['col'])
    plot_data2['method'] = 'hmc2'

    merge_plot_data = pd.concat([plot_data, plot_data2])
    return merge_plot_data

merge_plots(ps['beta'], particles['beta'])

,id,col,value,method
0,0,0,-0.674847,mcmc1
1,1,0,-0.881034,mcmc1
2,2,0,-0.816940,mcmc1
3,3,0,-0.387857,mcmc1
4,4,0,-0.019620,mcmc1
...,...,...,...,...
5995,995,5,0.517974,hmc2
5996,996,5,0.363894,hmc2
5997,997,5,0.323689,hmc2
5998,998,5,0.638105,hmc2


In [60]:
merge_chart = alt.Chart(merge_plots(ps['beta'], particles['beta'])).mark_line(
    opacity = 0.6,
    strokeWidth = 1,
    color='red'
).encode(
    alt.Y('value', title=None),
    alt.X('id:O',
          title=None
         ), 
    alt.Color('method')
).properties(width=200, height=100)

(merge_chart).facet(
    alt.Facet('col'),
    columns=3
)


alt.FacetChart(...)